In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor
%store -r agg_actions
%store -r editor_inputname
%store -r calculator

if ('the_page' not in locals() or 
    'the_editor' not in locals() or 
    'agg_actions' not in locals() or 
    'editor_inputname' not in locals() or 
    'calculator' not in locals()):
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))
    the_editor = pickle.load(open("data/the_editor.p",'rb'))
    agg_actions = pickle.load(open("data/agg_actions.p",'rb'))
    editor_inputname = pickle.load(open("data/editor_inputname.p",'rb'))
    calculator = pickle.load(open("data/calculator.p",'rb'))


from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. Detecting conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
elegible_actions = calculator.elegible_actions
conflicting_actions = calculator.get_conflicting_actions(editor_inputname)

from external.wikipedia import WikipediaDV, WikipediaAPI
wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
conflicting_editors = conflicting_actions['editor'].unique().tolist()

editors = wikipedia_dv.get_editors([int(x) for x in conflicting_editors if x[:2] != '0|'])
editors['userid'] = editors['userid'].astype('str')

full_editors_conflicts = calculator.get_conflict_score_per_editor()
editors[['userid','name','registration']].merge( full_editors_conflicts, 
         right_index=True, left_on='userid',how='left').set_index('userid')

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Activity of conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
editors['userid'] = editors['userid'].astype('int')
conf_editor_agg_actions = editors[['userid','name','registration']].merge(agg_actions, 
         left_on='userid', right_on='editor_id', how='left').set_index('userid')

# Listener
from visualization.actions_listener import ActionsListener
listener = ActionsListener(conf_editor_agg_actions)
actions = (conf_editor_agg_actions.loc[:,'total':'total_stopword_count'].columns.append(
    conf_editor_agg_actions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()

# Visualization
from ipywidgets import interact
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets.widgets import Dropdown

interact(listener.listen, 
         _range = get_date_slider_from_datetime(conf_editor_agg_actions['year_month']),
         editor=Dropdown(options=['All'] + editors['name'].values.tolist(), value='All'),
         granularity=Dropdown(options=['Yearly', 'Monthly'], value='Monthly'),
         black=Dropdown(options=actions, value='total'), 
         red=Dropdown(options= ['None'] + actions, value='total_surv_48h'),
         green=Dropdown(options= ['None'] + actions, value='None'), 
         blue=Dropdown(options= ['None'] + actions, value='None'))
         

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# C. Tokens of conflicting editors"))
display(md(f"***Page: {the_page['title']}***"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
editors['userid'] = editors['userid'].astype('str')
editor_conflicts = editors[['userid','name','registration']].merge(conflicting_actions, 
         left_on='userid', right_on='editor', how='left').set_index('userid')

#listener
from visualization.wordcloud_listener import SimpleWCListener
listener = SimpleWCListener(editor_conflicts)

# visualization
from utils.notebooks import get_date_slider_from_datetime
from ipywidgets import interact
from ipywidgets.widgets import Dropdown

interact(listener.listen, 
         _range=get_date_slider_from_datetime(editor_conflicts['rev_time']),
         action=Dropdown(options=['Both', 'Just Insertions', 'Just Deletions'], value='Both'),
         editor=Dropdown(options=['All']  + editors['name'].values.tolist(), value='All'),
         only_conflict = True)


In [ ]:
from IPython.display import HTML
from utils.notebooks import get_next_notebook

display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))
